In [1]:
import requests
import xlrd
import yaml
from xlutils.copy import copy
from tqdm import tqdm

In [2]:
def read_yaml(yaml_path):
    with open(yaml_path, encoding="utf-8", mode="r") as f:
        result = yaml.load(stream=f,Loader=yaml.FullLoader)
        return result

In [3]:
def excel(result):

    wb = xlrd.open_workbook(result['example_excel_path'])
    st = wb.sheet_by_index(0)  # sheet索引从0开始

    excel = copy(wb)

    write_excel = excel.get_sheet(0)

    return excel, write_excel, st

In [4]:
def create_dict(jcid):
    url = 'https://s-file-1.ykt.cbern.com.cn/zxx/ndrs/prepare_lesson/trees/{}.json'.format(jcid)
    responce = requests.get(url).json()
    my_dict = {}
    for i in range(0,len(responce)):
        my_dict[responce[i]['id']] = responce[i]['title']
        try:
            len(responce[i]['child_nodes'])
        except:
            continue
        for j in range(0,len(responce[i]['child_nodes'])):
            my_dict[responce[i]['child_nodes'][j]['id']] = responce[i]['child_nodes'][j]['title']
            try:
                len(responce[i]['child_nodes'][j]['child_nodes'])
            except:
                continue
            for k in range(0,len(responce[i]['child_nodes'][j]['child_nodes'])):
                my_dict[responce[i]['child_nodes'][j]['child_nodes'][k]['id']] = responce[i]['child_nodes'][j]['child_nodes'][k]['title']
                try:
                    len(responce[i]['child_nodes'][j]['child_nodes'][k]['child_nodes'])
                except:
                    continue
                for l in range(0,len(responce[i]['child_nodes'][j]['child_nodes'][k]['child_nodes'])):
                    my_dict[responce[i]['child_nodes'][j]['child_nodes'][k]['child_nodes'][l]['id']] = responce[i]['child_nodes'][j]['child_nodes'][k]['child_nodes'][l]['title']
    return my_dict            
    

In [ ]:
dt=create_dict('2fbcdb5d-0682-4cca-b979-076d0119e3d3')
dt['d51dc441-7630-3a6b-b9da-21316e49b7a3']

In [5]:
def main(msgs, jcid, write_excel, row):
    
    url = "https://s-file-1.ykt.cbern.com.cn/zxx/ndrs/prepare_lesson/teachingmaterials/{}/resources/part_100.json".format(jcid)

    responce = requests.get(url).json()

    my_dict = create_dict(jcid)#目录编码对应的字典
    
    for i in range(0,len(responce)):
        try:
            rel = list(responce[i]['relations'].keys())
        except:
            continue
        for r in rel:
            for j in range(0,len(responce[i]['relations'][r])):
                try:
                    responce[i]['relations'][r][j]['resource_type_code']
                except:
                    continue
                if responce[i]['relations'][r][j]['resource_type_code'] == "lesson_plandesign":
                    link_id = responce[i]['relations'][r][j]['id']
                    flag = 0
                    url1 = "https://s-file-1.ykt.cbern.com.cn/zxx/ndrv2/prepare_sub_type/resources/details/{}.json".format(link_id)

                    responce1 = requests.get(url1).json()
                    try:
                        for d in range(len(responce1['ti_items'][1]['ti_storages'])):
                            if responce1['ti_items'][d]['ti_file_flag'] == 'source':   
                                flag = 1 
                                dlink = responce1['ti_items'][d]['ti_storages'][0].replace('-private', '')
                                break
                    except:
                         continue
                    
                    if flag == 0:
                        continue

                    write_excel.write(row,9,dlink)

                    write_excel.write(row,0,'国家中小学智慧教育平台')
                    write_excel.write(row,1,msgs[0])
                    write_excel.write(row,2,msgs[1])
                    write_excel.write(row,3,msgs[2])
                    write_excel.write(row,4,msgs[3])
                    for k in range(0,len(responce[i]['chapter_ids'])):
                        write_excel.write(row,5+k,my_dict[responce[i]['chapter_ids'][k]])#保存目录
                    row += 1
                    break
                else:
                    continue         
    return row

In [6]:
if __name__ == '__main__':

    result = read_yaml("../../settings/settings.yaml")

    jc_st = xlrd.open_workbook(result['teaching_jcb_excel_path']).sheet_by_index(0)

    new_excel, write_excel, st = excel(result)

    row = st.nrows

    # msgs = ['小学','语文','统编版','一年级下册']


    for l in tqdm(range(1, jc_st.nrows)):
        #对于每一行，使用列表推导式将该行的每个单元格的值转换为字符串，并将这些字符串值存储在 lie 列表中
        lie = [str(jc_st.cell_value(l, i)) for i in range(0, jc_st.ncols)]
        
        row = main(lie[0:4], lie[4], write_excel, row)
            
    new_excel.save(result['teaching_plan_excel_path'])
    

100%|██████████| 1871/1871 [7:04:43<00:00, 13.62s/it]   


In [ ]:
new_excel.save(result['teaching_plan_excel_path'])